In [164]:
#libraries
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine
from sklearn.preprocessing import normalize

In [165]:
#read the data & have randome sample for reproducability
recipes = pd.read_csv('../data/full_clean_data.csv')

In [166]:
##rename the dataframe index
recipes.index.names = ['recipes_id']
##convert index to column
recipes['recipes_id'] = recipes.index
recipes.head(1)

,name,url,category,rating,rating_count,ingredients,total,servings,yield,calories,carbohydrates_g,fat_g,protein_g,ingredients_parsed,recipes_id
recipes_id,,,,,,,,,,,,,,,
0,Dessert Crepes,https://www.allrecipes.com/recipe/19037/desser...,breakfast-and-brunch,4.8,1156,"['4 eggs, lightly beaten ', ' 1\u2009⅓ cups m...",20,8,8 crepes,163.8,17.2,7.7,6.4,"['milk', 'flour', 'sugar']",0


In [167]:
##normalize the numerical column (total+nutritional values) in new dataframe << because i dont want to normalize all other features
newrecipes = recipes[['total', 'calories', 'carbohydrates_g', 'fat_g', 'protein_g']]
recipes_norm = pd.DataFrame(normalize(newrecipes, axis=0))
recipes_norm.columns = newrecipes.columns
recipes_norm.index = newrecipes.index
recipes_norm.head(2)

,total,calories,carbohydrates_g,fat_g,protein_g
recipes_id,,,,,
0,0.000837,0.003888,0.003539,0.003818,0.002726
1,0.002512,0.011174,0.005103,0.012346,0.017931


In [168]:
#####creare a function that recommends top 3 similar recipe using cosine. Guidance reference https://www.kaggle.com/code/yyzz1010/content-based-filtering-recipe-recommender?scriptVersionId=21219821&cellId=23
def recipe_recommender(recipes_id):
    
     allRecipes = pd.DataFrame(recipes_norm.index)
     allRecipes = allRecipes[allRecipes.recipes_id != recipes_id]
     allRecipes["distance"] = allRecipes["recipes_id"].apply(lambda x: cosine(recipes_norm.loc[recipes_id], recipes_norm.loc[x]))
     # sort by distance then recipe id, the smaller value of recipe id will be picked.
     Top3NRecommendation = allRecipes.sort_values(["distance"]).head(3).sort_values(by=['distance', 'recipes_id'], ascending=False)
     #print the distance value
     print(Top3NRecommendation)
     #create a list of the recommended recipe output
     recipes_id = [recipes_id]
     recipe_list = [] #'name', 'url', 'total cooking time', 'cal', 'carbs', 'fat','protein'
     for recipeid in Top3NRecommendation.recipes_id:
        recipes_id.append(recipeid)   # list of recipe id of selected recipe and recommended recipe
        recipe_list.append("{}  {}".format(recipeid, recipes.at[recipeid, 'name']))
        recipe_list.append("{}  {}".format(recipeid, recipes.at[recipeid, 'url']))
        recipe_list.append("{}  {}".format(recipeid, recipes.at[recipeid, 'total']))
        recipe_list.append("{}  {}".format(recipeid, recipes.at[recipeid, 'calories']))
        recipe_list.append("{}  {}".format(recipeid, recipes.at[recipeid, 'carbohydrates_g']))
        recipe_list.append("{}  {}".format(recipeid, recipes.at[recipeid, 'fat_g']))
        recipe_list.append("{}  {}".format(recipeid, recipes.at[recipeid, 'protein_g']))
     return recipe_list

In [169]:
recipes[recipes['name'] == 'Potato Waffles']

,name,url,category,rating,rating_count,ingredients,total,servings,yield,calories,carbohydrates_g,fat_g,protein_g,ingredients_parsed,recipes_id
recipes_id,,,,,,,,,,,,,,,
4594,Potato Waffles,https://www.allrecipes.com/recipe/220890/potat...,breakfast-and-brunch,4.57,112,"['2 tablespoons butter ', ' 1 onion, chopped ...",25,4,4 servings,216.7,27.9,9.0,6.5,"['butter', 'mashed potato', 'flour', 'egg']",4594


In [170]:
#recommend the top 3 most similar (total, calories, protein, carbs, fat)
recipe_recommender(4594)

       recipes_id  distance
637           637  0.000232
3319         3319  0.000206
16689       16689  0.000176


['637  Quick Noodle-Veggie Bowls',
 '637  https://www.allrecipes.com/recipe/283693/quick-noodle-veggie-bowls/',
 '637  20',
 '637  194.4',
 '637  25.0',
 '637  8.4',
 '637  5.8',
 '3319  Caramelized French Toast',
 '3319  https://www.allrecipes.com/recipe/24571/caramelized-french-toast/',
 '3319  30',
 '3319  247.5',
 '3319  31.4',
 '3319  10.6',
 '3319  7.2',
 '16689  Easy Butternut Squash Ravioli',
 '16689  https://www.allrecipes.com/recipe/219174/easy-butternut-squash-ravioli/',
 '16689  40',
 '16689  377.7',
 '16689  47.4',
 '16689  15.9',
 '16689  11.5']